# there is the SHAPE dataset of 50 sequences, under 4 conditions... 

we want to see of our accessibility is good

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

<IPython.core.display.HTML object>

# Load data

In [18]:
import simushape as ss 
from scipy.stats import spearmanr as spear
import pandas
import numpy as np

def run_access(data, keys):
    shaker_shape = ss.crosspredict(data,keys)
    seqs = [data[k][1] for k in keys] 
    acc_shaker = [rnaplfold(seq, rea) for seq,rea in zip(seqs,shaker_shape)]
    acc_nothing = [rnaplfold(seq) for seq in seqs]
    acc_real  = [rnaplfold(data[k][1],data[k][0]) for k in keys]
    corr = lambda other: [spear(accr, acco)[0] for accr,acco in zip(acc_real, other)]
    corr_shaker = corr(acc_shaker)
    corr_default = corr(acc_nothing)
    
    index= ['shaker', 'default']
    data = [corr_shaker,corr_default]
    map(lambda x:x.append(np.mean(x)),data)
    print pandas.DataFrame(data, columns=keys+['mean'], index=index).T
    

In [ ]:
import rna_tools.rna_io as rio


# TODO: add 4th

types = ["cellfree",
"incell",
"kasugamycin"]

    

def getdata(typ):
    return rio.get_all_data("../data/weeks194_orig/%s.react" % typ,"../data/weeks194_orig/%s.dbn" % typ)  # {key: rea, seq, stru}

data = getdata(types[2]) # 2 has bad results :) 

keys = data.keys()[:3]

run_access(data,keys)

#for k,v in data.items():print k,"\t",  len(v[1]), len(v[0])

crosspredict:

 .

ImportError: No module named line_profiler

# get

#   

                                     shaker   default
metG_74                            0.898258  1.000000
topA_54                            0.797323  0.871321
rpoS_102                           0.824116  0.858536
cspE_28                            0.915294  0.762164
serS_41                            0.808504  0.936856
sodB_60                            0.755673  0.770831
rpmI_rplT_63                       0.889860  0.894791
dcuA_182                           0.759789  0.834384
lptF_lptG_192                      0.804614  0.902440
gnd_73                             0.778166  0.883780
oppA_oppB_oppC_oppD_oppF_53        0.600823  0.804871
pheS_pheT_ihfA_62                  0.834021  0.921648
ribF_ileS_lspA_fkpB_ispH_6         0.811495  0.849922
yifL_dapF_yigA_xerC_yigB_uvrD_159  0.865668  0.968483
tRNA_glyW_tRNA_cysT_tRNA_leuZ_72   0.855657  0.921214
mean                               0.813284  0.878750

(1, '', '')

In [14]:
"%.100f" % 0.00000001

'0.0000000100000000000000002092256083012847267532663408928783610463142395019531250000000000000000000000'